# DataFrames avec PySpark : comparaison avec les RDD

Ce notebook complète le **tuto RDD** en présentant les **DataFrames** dans PySpark, et en comparant les deux approches sur un même exemple (*Word Count*).

Objectifs :
- Comprendre la différence **RDD vs DataFrame**.
- Créer des **DataFrames** à partir de collections locales et de fichiers.
- Manipuler les DataFrames : `select`, `filter`, `withColumn`, `groupBy`, `agg`, `orderBy`.
- Implémenter un **Word Count** avec DataFrames.
- Comparer Word Count **RDD vs DataFrame** (lisibilité, optimisations, API).


In [1]:
from pyspark.sql import SparkSession

# Création d'une SparkSession locale
spark = SparkSession.builder.appName("TutoDataFrame_PySpark").master("local[*]").getOrCreate()

# SparkContext, pour la partie comparaison RDD
sc = spark.sparkContext

print(spark)
print(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/19 05:25:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/19 05:25:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/19 05:25:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


<SparkContext master=local[*] appName=TutoDataFrame_PySpark>


## 1. DataFrame vs RDD : rappel conceptuel

Un **RDD** est :

- une collection distribuée d'objets Python,
- sans schéma explicite,
- manipulée avec des fonctions Python (lambda) côté driver,
- peu optimisable automatiquement.

Un **DataFrame** est :

- une **table distribuée** avec un **schéma** (colonnes typées),
- manipulée avec une API déclarative (semblable à SQL),
- traduite en un **plan logique** puis optimisée par le **Catalyst optimizer** de Spark,
- exécutée avec des optimisations bas niveau (projet Tungsten).

En pratique, en PySpark moderne :

- on privilégie les **DataFrames** (et Spark SQL) pour la plupart des traitements,
- on réserve les **RDD** à des cas très spécifiques (API bas niveau, transformations très personnalisées, compatibilité ancienne).

In [3]:
data = list(range(1, 6))

#  RDD
rdd_numbers = sc.parallelize(data)

# DataFrame
df_numbers = spark.createDataFrame([(x,) for x in data], ["value"])

print("RDD :", rdd_numbers.collect())
df_numbers.show()

RDD : [1, 2, 3, 4, 5]
+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    4|
|    5|
+-----+



## 2. Créer des DataFrames

On peut créer un DataFrame de plusieurs façons :

1. **À partir d'une collection locale** (liste de tuples, liste de dictionnaires, etc.) avec `spark.createDataFrame(...)`.
2. **À partir d'un fichier** avec les lecteurs intégrés : `spark.read.csv`, `spark.read.json`, `spark.read.parquet`, `spark.read.table`, etc.


In [4]:
# 2.1. À partir d'une liste de dictionnaires

data_people = [
    {"id": 1, "name": "Alice", "age": 25},
    {"id": 2, "name": "Bob", "age": 30},
    {"id": 3, "name": "Charlie", "age": 35},
]

df_people = spark.createDataFrame(data_people)
df_people.printSchema()
df_people.show()

root
 |-- age: long (nullable = true)
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)

+---+---+-------+
|age| id|   name|
+---+---+-------+
| 25|  1|  Alice|
| 30|  2|    Bob|
| 35|  3|Charlie|
+---+---+-------+



In [5]:
# 2.2. À partir d'une liste de tuples + noms de colonnes

data_sales = [
    (1, "Paris", "A", 10, 100.0),
    (2, "Lyon", "A", 5, 100.0),
    (3, "Paris", "B", 8, 80.0),
]

cols = ["id", "city", "product", "quantity", "unit_price"]

df_sales = spark.createDataFrame(data_sales, cols)
df_sales.printSchema()
df_sales.show()

root
 |-- id: long (nullable = true)
 |-- city: string (nullable = true)
 |-- product: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- unit_price: double (nullable = true)

+---+-----+-------+--------+----------+
| id| city|product|quantity|unit_price|
+---+-----+-------+--------+----------+
|  1|Paris|      A|      10|     100.0|
|  2| Lyon|      A|       5|     100.0|
|  3|Paris|      B|       8|      80.0|
+---+-----+-------+--------+----------+



### 2.3. Lecture d'un fichier CSV

Exemple de lecture d'un fichier CSV (à adapter à ton contexte) :

```python
df_csv = spark.read.csv("ventes.csv", header=True, inferSchema=True)
df_csv.show()
```

## 3. Opérations de base sur les DataFrames

Quelques opérations classiques :

- `select` : sélectionner des colonnes.
- `filter` / `where` : filtrer les lignes.
- `withColumn` : ajouter / transformer une colonne.
- `groupBy(...).agg(...)` : agrégations par groupe.
- `orderBy` / `sort` : trier le résultat.


In [6]:
from pyspark.sql.functions import col, expr

df_sales.show()

# SELECT city, product, quantity
df_sel = df_sales.select("city", "product", "quantity")
df_sel.show()

# WHERE quantity > 6
df_filtered = df_sales.filter(col("quantity") > 6)
df_filtered.show()

# withColumn : ajouter une colonne 'revenue' = quantity * unit_price
df_revenue = df_sales.withColumn("revenue", col("quantity") * col("unit_price"))
df_revenue.show()

# groupBy city : somme du revenue par ville
df_city_revenue = df_revenue.groupBy("city").agg(expr("sum(revenue) as total_revenue"))
df_city_revenue.show()

# Trier par revenue décroissant
df_city_revenue.orderBy(col("total_revenue").desc()).show()

+---+-----+-------+--------+----------+
| id| city|product|quantity|unit_price|
+---+-----+-------+--------+----------+
|  1|Paris|      A|      10|     100.0|
|  2| Lyon|      A|       5|     100.0|
|  3|Paris|      B|       8|      80.0|
+---+-----+-------+--------+----------+

+-----+-------+--------+
| city|product|quantity|
+-----+-------+--------+
|Paris|      A|      10|
| Lyon|      A|       5|
|Paris|      B|       8|
+-----+-------+--------+

+---+-----+-------+--------+----------+
| id| city|product|quantity|unit_price|
+---+-----+-------+--------+----------+
|  1|Paris|      A|      10|     100.0|
|  3|Paris|      B|       8|      80.0|
+---+-----+-------+--------+----------+

+---+-----+-------+--------+----------+-------+
| id| city|product|quantity|unit_price|revenue|
+---+-----+-------+--------+----------+-------+
|  1|Paris|      A|      10|     100.0| 1000.0|
|  2| Lyon|      A|       5|     100.0|  500.0|
|  3|Paris|      B|       8|      80.0|  640.0|
+---+-----+---

## 4. Transformations DataFrame vs RDD

Quelques parallèles :

- **RDD** : `map(lambda x: ...)`  
  **DF**  : `select(expr(...))`, `withColumn(...)`

- **RDD** : `filter(lambda x: condition)`  
  **DF**  : `filter(condition)` ou `where(condition)`

- **RDD** : RDD de paires `(key, value)` + `reduceByKey`  
  **DF**  : `groupBy("key").agg(...)`

- **RDD** : `sortBy(...)`  
  **DF**  : `orderBy(...)` / `sort(...)`

L'idée générale :  
> Avec les DataFrames, on écrit **ce qu'on veut obtenir**,  
> Spark se charge de **comment l'exécuter efficacement** (optimiseur Catalyst).

## 5. Word Count avec DataFrames

On va maintenant implémenter **Word Count** avec DataFrames, en réutilisant le même fichier texte `data.txt` que dans le notebook RDD.

### Étapes (version DataFrame) :

1. Lire le fichier texte avec `spark.read.text("data.txt")` → une colonne `value`.
2. Nettoyer le texte (optionnel mais utile) : mettre en minuscule, enlever la ponctuation.
3. Découper la colonne `value` en mots (`split`), puis exploser en lignes (`explode`).
4. Filtrer les mots vides.
5. `groupBy("word").count()`.
6. Trier par `count` décroissant et afficher les N premiers mots.


In [8]:
from pyspark.sql.functions import split, explode, lower, regexp_replace, desc

# 1. Lecture du fichier texte
# Adapte le chemin à ton environnement, comme dans le notebook RDD
df_lines = spark.read.text("data.txt")

df_lines.show(5, truncate=False)

+----------------------------------------------------------------------------------------+
|value                                                                                   |
+----------------------------------------------------------------------------------------+
|Spark est un moteur de calcul distribué pour le big data.                               |
|Le big data désigne des volumes de données trop importants pour un traitement classique.|
|Avec Spark, on peut traiter des données en mémoire de manière très efficace.            |
|Un RDD est une collection distribuée, immuable et tolérante aux pannes.                 |
|Les DataFrames offrent une API de plus haut niveau que les RDD.                         |
+----------------------------------------------------------------------------------------+
only showing top 5 rows



In [9]:
# 2. Nettoyage de base : 
# - mise en minuscule
# - suppression des caractères de ponctuation (remplacés par un espace)

df_clean = df_lines.select(
    lower(
        regexp_replace(col("value"), r"[^\w\s]", " ")
    ).alias("line")
)

df_clean.show(5, truncate=False)

+----------------------------------------------------------------------------------------+
|line                                                                                    |
+----------------------------------------------------------------------------------------+
|spark est un moteur de calcul distribu  pour le big data                                |
|le big data d signe des volumes de donn es trop importants pour un traitement classique |
|avec spark  on peut traiter des donn es en m moire de mani re tr s efficace             |
|un rdd est une collection distribu e  immuable et tol rante aux pannes                  |
|les dataframes offrent une api de plus haut niveau que les rdd                          |
+----------------------------------------------------------------------------------------+
only showing top 5 rows



In [10]:
# 3. Split + explode pour obtenir une ligne par mot

df_words = df_clean.select(
    explode(
        split(col("line"), "\s+")
    ).alias("word")
)

df_words.show(10)

+--------+
|    word|
+--------+
|   spark|
|     est|
|      un|
|  moteur|
|      de|
|  calcul|
|distribu|
|    pour|
|      le|
|     big|
+--------+
only showing top 10 rows



In [11]:
# 4. Filtrer les chaînes vides
df_words_non_empty = df_words.filter(col("word") != "")

# 5. groupBy + count
df_word_counts = df_words_non_empty.groupBy("word").count()

# 6. Tri décroissant sur count
df_top_words = df_word_counts.orderBy(desc("count"))

df_top_words.show(20)

+-----+-----+
| word|count|
+-----+-----+
|  les|   30|
|   de|   21|
|   le|   18|
|    d|   14|
| pour|   13|
|spark|   13|
|   et|   13|
|    r|   12|
|   un|   12|
|  des|   11|
|   es|   10|
| avec|   10|
|  rdd|    9|
| peut|    8|
| dans|    8|
|count|    7|
|   en|    7|
| sont|    7|
|    s|    7|
|   ou|    7|
+-----+-----+
only showing top 20 rows



## 6. Comparaison avec Word Count en RDD

Rappel : la version RDD du Word Count ressemble à ceci :

```python
lines = sc.textFile("data.txt")
words = lines.flatMap(lambda line: line.split())
pairs = words.map(lambda w: (w, 1))
word_counts = pairs.reduceByKey(lambda a, b: a + b)
sorted_counts = word_counts.sortBy(lambda kv: kv[1], ascending=False)
sorted_counts.take(20)
```

On va réexécuter une version RDD ici pour comparer le code et les résultats.


In [ ]:
# Word Count version RDD 

lines_rdd = sc.textFile("data.txt")
words_rdd = lines_rdd.flatMap(lambda line: line.split())

pairs_rdd = words_rdd.map(lambda w: (w, 1))
word_counts_rdd = pairs_rdd.reduceByKey(lambda a, b: a + b)
sorted_counts_rdd = word_counts_rdd.sortBy(lambda kv: kv[1], ascending=False)

top20_rdd = sorted_counts_rdd.take(20)
top20_rdd

[('de', 21),
 ('les', 19),
 ('le', 14),
 ('et', 13),
 ('pour', 13),
 ('des', 11),
 ('Les', 11),
 ('avec', 9),
 ('un', 8),
 ('peut', 8),
 ('RDD', 8),
 ('sont', 7),
 ('Spark', 7),
 ('en', 7),
 ('ou', 7),
 ('est', 6),
 ('count', 6),
 ('sur', 5),
 ('fichiers', 5),
 ('données', 5)]

## 7. Discussion : RDD vs DataFrame sur Word Count

### 7.1 Lisibilité du code

- **RDD** : on manipule des tuples `(mot, 1)` et des lambdas Python.
- **DataFrame** : on manipule des colonnes nommées (`word`, `count`) avec des opérations de type SQL (`groupBy`, `count`, `orderBy`).

La version DataFrame est souvent plus **déclarative** et plus simple à lire pour quelqu'un habitué à SQL / BI.

### 7.2 Optimisations

- Les DataFrames passent par l'**optimiseur Catalyst**.
- Spark peut **réorganiser le plan**, **pousser les filtres**, **optimiser les agrégations**, etc.
- Avec des RDD, Spark voit seulement des fonctions Python opaques → **moins d'optimisations automatiques**.

### 7.3 Typage et schéma

- RDD : pas de schéma global, juste des objets Python.
- DataFrame : schéma explicite, facilité d'intégration avec des outils SQL (Spark SQL, JDBC, BI, etc.).

### 7.4 Quand utiliser quoi ?

- **DataFrames** (recommandé dans la majorité des cas) :
  - traitements de données structurées / semi-structurées,
  - jointures, agrégations, pipelines de transformations,
  - intégration avec Spark SQL, MLlib (DataFrame API), etc.

- **RDD** :
  - besoin de transformations très bas niveau / personnalisées,
  - besoin d'une API fonctionnelle proche de l'historique de Spark,
  - cas spécifiques non bien couverts par l'API DataFrame.

En PySpark moderne, la bonne pratique est :  
> Commencer par **DataFrames**, et ne descendre vers les **RDD** que si nécessaire.


## 8. Conclusion

Dans ce notebook, tu as :

- créé des **DataFrames** à partir de listes Python et de texte,
- manipulé des DataFrames avec `select`, `filter`, `withColumn`, `groupBy`, `orderBy`,
- implémenté un **Word Count** avec DataFrames,
- comparé cette approche avec la version **RDD**.

N'oublie pas d'arrêter la session Spark une fois terminé :


In [13]:
spark.stop()